## Lendo csv

In [1]:
import pandas as pd
credit_data = pd.read_csv('../dataset/credit_data.csv')

## Localizar problemas

In [2]:
## Localizar 
credit_data.loc[credit_data['age'] < 0]
credit_data.loc[pd.isnull(credit_data['age'])]


## Preencher os valores com a média
credit_data.loc[credit_data.age < 0, 'age'] = credit_data['age'][credit_data.age > 0].mean()
credit_data.fillna(credit_data.mean(), inplace=True)

previsores = credit_data.iloc[:, 1:4].values
classe = credit_data.iloc[:, 4].values

## Padronizar dados

In [3]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

## Divisão do dataset

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

def dataset_division(previsores, classse):
    previsores_train, previsores_test, classe_train, classe_test = train_test_split(previsores, classe, test_size=0.25, random_state=0)
    return previsores_train, previsores_test, classe_train, classe_test

## Validação Cruzada

In [5]:
from sklearn.model_selection import cross_val_score

def cross_validation(model, previsores, classe):
    
    resultado = cross_val_score(Naive_Bayes, previsores, classe, cv = 10)
    return resultado.mean()
    

## Validação Cruzada Stratified Fold

In [6]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

def stratified_fold(model, previsores, classe, seed):
    a = np.zeros(5)
    b = np.zeros(shape=(previsores.shape[0], 1))


    kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = seed)

    resultados = []
    matrizes = []
    for indice_treinamento, indice_teste in kfold.split(previsores, np.zeros(shape=(previsores.shape[0], 1))):

        model.fit(previsores[indice_treinamento], classe[indice_treinamento])
        previsoes = model.predict(previsores[indice_teste])
        precisao = accuracy_score(classe[indice_teste], previsoes)
        matrizes.append(confusion_matrix(classe[indice_teste], previsoes))
        resultados.append(precisao)

    matriz_final = np.mean(matrizes, axis = 0)
    resultados = np.asarray(resultados)
    
    return resultados.mean(), matriz_final

## Salvar modelos

In [7]:
import pickle
def save_model(model, name):
    pickle.dump(model, open('../save-models/' + name + '.sav', 'wb'))

## Carregar modelos

In [8]:
def load_model(name):
    return pickle.load(open('../save-models/' + name + '.sav', 'rb'))

## Naive Bayes -> Tabela de Probabilidade

In [9]:
from sklearn.naive_bayes import GaussianNB
Naive_Bayes = GaussianNB()

##Divisao normal do dataset

'''
previsores_train, previsores_test, classe_train, classe_test = dataset_division(previsores, classe)
Naive_Bayes.fit(previsores_train, classe_train)
previsoes = Naive_Bayes.predict(previsores_test)
Naive_Bayes_accuracy = accuracy_score(classe_test, previsoes)
matriz = confusion_matrix(classe_test, previsoes)
'''

##Cross Validation
'''
Naive_Bayes_accuracy = cross_validation(Naive_Bayes, previsores, classe)
'''

##Stratified Fold
Naive_Bayes_accuracy, _ = stratified_fold(Naive_Bayes, previsores, classe, 0)
save_model(Naive_Bayes, 'Naive_Bayes')

## Árvore de Decisão

In [10]:
from sklearn.tree import DecisionTreeClassifier
Decision_Tree = DecisionTreeClassifier(criterion='entropy', random_state=0)

##Divisao normal do dataset

'''
previsores_train, previsores_test, classe_train, classe_test = dataset_division(previsores, classe)
Decision_Tree.fit(previsores_train, classe_train)
previsoes = Decision_Tree.predict(previsores_test)
Decision_Tree_accuracy = accuracy_score(classe_test, previsoes)
matriz = confusion_matrix(classe_test, previsoes)
'''

##Cross Validation
'''
Decision_Tree_accuracy = cross_validation(Decision_Tree, previsores, classe)
'''

##Stratified Fold
Decision_Tree_accuracy, _ = stratified_fold(Decision_Tree, previsores, classe, 0)
save_model(Decision_Tree, 'Decision_Tree')

## Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier
Random_Forest = RandomForestClassifier(n_estimators = 40,
                                      criterion = 'entropy',
                                      random_state = 0)

##Divisao normal do dataset

'''
previsores_train, previsores_test, classe_train, classe_test = dataset_division(previsores, classe)
Random_Forest.fit(previsores_train, classe_train)
previsoes = Random_Forest.predict(previsores_test)
Random_Forest_accuracy = accuracy_score(classe_test, previsoes)
matriz = confusion_matrix(classe_test, previsoes)
'''

##Cross Validation
'''
Random_Forest_accuracy = cross_validation(Random_Forest, previsores, classe)
'''

##Stratified Fold
Random_Forest_accuracy, _ = stratified_fold(Random_Forest, previsores, classe, 0)
save_model(Random_Forest, 'Random_Forest')

## Regras -> CN2

In [13]:
import Orange

base = Orange.data.Table('../dataset/credit_data.csv')
print(base.domain)

base_dividida = Orange.evaluation.testing.sample(base, n=0.25)
base_treinamento = base_dividida[1]
base_teste = base_dividida[0]

cn2_learner = Orange.classification.rules.CN2Learner()
classificador = cn2_learner(base_treinamento)

for regras in classificador.rule_list:
    print(regras)
    
resultado = Orange.evaluation.testing.TestOnTestData(base_treinamento, base_teste, [classificador])

[income, age, loan | default]
IF age>=34.9966813726351 THEN default=0 
IF loan<=2495.13299137587 AND income>=20145.9885970689 THEN default=0 
IF income<=34145.7995516248 AND loan>=4096.783714405129 THEN default=1 
IF loan>=7708.31562497011 AND loan>=9698.582169129 THEN default=1 
IF loan>=7708.31562497011 AND loan>=9595.28628892989 THEN default=0 
IF loan>=7708.31562497011 AND age>=20.9909665295854 THEN default=1 
IF income>=58132.4712652713 AND age>=20.3008601283655 THEN default=0 
IF income<=22089.8374845274 AND age>=23.238169600255798 THEN default=1 
IF loan<=3293.25087871026 AND income>=26617.0303151011 THEN default=0 
IF age<=18.4162362311035 AND income>=42522.5757574663 THEN default=0 
IF loan<=5836.56338145928 AND loan>=5836.56338145928 THEN default=1 
IF loan<=5862.83302915672 AND loan>=5862.83302915672 THEN default=1 
IF loan<=5883.660557890999 AND income>=45360.716274008206 THEN default=0 
IF loan>=4849.33378471958 AND age>=26.854012909811 THEN default=1 
IF loan>=4849.333784

## Majority Learner

In [15]:
import Orange

base = Orange.data.Table('../dataset/credit_data.csv')
print(base.domain)

base_dividida = Orange.evaluation.testing.sample(base, n=0.25)
base_treinamento = base_dividida[1]
base_teste = base_dividida[0]

classificador = Orange.classification.MajorityLearner()
resultado = Orange.evaluation.testing.TestOnTestData(base_treinamento, base_teste, [classificador])
print('Acurácia: %.2f%%' % Orange.evaluation.CA(resultado))

[income, age, loan | default]
Acurácia: 0.86%


## Base Line Classifier

In [16]:
## O minimo que o modelo necessita aprender

import collections
previsores_train, previsores_test, classe_train, classe_test = dataset_division(previsores, classe)
counter = collections.Counter(classe_test)
base_line = counter[0] / (counter[0] + counter[1])

## KNN

In [17]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)

##Divisao normal do dataset

'''
previsores_train, previsores_test, classe_train, classe_test = dataset_division(previsores, classe)
KNN.fit(previsores_train, classe_train)
previsoes = KNN.predict(previsores_test)
KNN_accuracy = accuracy_score(classe_test, previsoes)
matriz = confusion_matrix(classe_test, previsoes)
'''

##Cross Validation
'''
KNN_accuracy = cross_validation(KNN, previsores, classe)
'''

##Stratified Fold
KNN_accuracy, _ = stratified_fold(KNN, previsores, classe, 0)
save_model(KNN, 'KNN')

## Regressão Logística

In [19]:
from sklearn.linear_model import LogisticRegression
Logistic_Regression = LogisticRegression(random_state = 0)

##Divisao normal do dataset

'''
previsores_train, previsores_test, classe_train, classe_test = dataset_division(previsores, classe)
Logistic_Regression.fit(previsores_train, classe_train)
previsoes = Logistic_Regression.predict(previsores_test)
Logistic_Regression_accuracy = accuracy_score(classe_test, previsoes)
matriz = confusion_matrix(classe_test, previsoes)
'''

##Cross Validation
'''
Logistic_Regression_accuracy = cross_validation(Logistic_Regression, previsores, classe)
'''

##Stratified Fold
Logistic_Regression_accuracy, _ = stratified_fold(Logistic_Regression, previsores, classe, 0)
save_model(Logistic_Regression, 'Logistic_Regression')

## SVM

In [38]:
from sklearn.svm import SVC
SVM = SVC(kernel = 'rbf', random_state = 0, C = 2.0, probability = True)

##Divisao normal do dataset

'''
previsores_train, previsores_test, classe_train, classe_test = dataset_division(previsores, classe)
SVM.fit(previsores_train, classe_train)
previsoes = SVM.predict(previsores_test)
SVM_accuracy = accuracy_score(classe_test, previsoes)
matriz = confusion_matrix(classe_test, previsoes)
'''

##Cross Validation
'''
SVM_accuracy = cross_validation(SVM, previsores, classe)
'''

##Stratified Fold
SVM_accuracy, _ = stratified_fold(SVM, previsores, classe, 0)
save_model(SVM, 'SVM')

## Redes Neurais com SkLearn

In [21]:
from sklearn.neural_network import MLPClassifier
MLP = MLPClassifier(verbose = True,
                    max_iter = 1000,
                    tol = 0.000010,
                    solver = 'adam',
                    hidden_layer_sizes = (100),
                    activation = 'relu')

##Divisao normal do dataset

'''
previsores_train, previsores_test, classe_train, classe_test = dataset_division(previsores, classe)
MLP.fit(previsores_train, classe_train)
previsoes = MLP.predict(previsores_test)
MLP_accuracy = accuracy_score(classe_test, previsoes)
matriz = confusion_matrix(classe_test, previsoes)
'''

##Cross Validation
'''
MLP_accuracy = cross_validation(MLP, previsores, classe)
'''

##Stratified Fold
MLP_accuracy, _ = stratified_fold(MLP, previsores, classe, 0)
save_model(MLP, 'MLP')

Iteration 1, loss = 0.71882549
Iteration 2, loss = 0.64142052
Iteration 3, loss = 0.57786543
Iteration 4, loss = 0.52504032
Iteration 5, loss = 0.48107012
Iteration 6, loss = 0.44368727
Iteration 7, loss = 0.41148265
Iteration 8, loss = 0.38262606
Iteration 9, loss = 0.35717871
Iteration 10, loss = 0.33413371
Iteration 11, loss = 0.31346916
Iteration 12, loss = 0.29456589
Iteration 13, loss = 0.27720247
Iteration 14, loss = 0.26148111
Iteration 15, loss = 0.24726715
Iteration 16, loss = 0.23406291
Iteration 17, loss = 0.22243152
Iteration 18, loss = 0.21138011
Iteration 19, loss = 0.20172439
Iteration 20, loss = 0.19274245
Iteration 21, loss = 0.18443426
Iteration 22, loss = 0.17709882
Iteration 23, loss = 0.17033604
Iteration 24, loss = 0.16407959
Iteration 25, loss = 0.15832050
Iteration 26, loss = 0.15305272
Iteration 27, loss = 0.14837154
Iteration 28, loss = 0.14387421
Iteration 29, loss = 0.13984719
Iteration 30, loss = 0.13610756
Iteration 31, loss = 0.13260383
Iteration 32, los

Iteration 272, loss = 0.02256948
Iteration 273, loss = 0.02244697
Iteration 274, loss = 0.02236782
Iteration 275, loss = 0.02235175
Iteration 276, loss = 0.02221423
Iteration 277, loss = 0.02214499
Iteration 278, loss = 0.02205377
Iteration 279, loss = 0.02204580
Iteration 280, loss = 0.02191850
Iteration 281, loss = 0.02189174
Iteration 282, loss = 0.02173431
Iteration 283, loss = 0.02169737
Iteration 284, loss = 0.02162650
Iteration 285, loss = 0.02151730
Iteration 286, loss = 0.02147292
Iteration 287, loss = 0.02141405
Iteration 288, loss = 0.02128757
Iteration 289, loss = 0.02124430
Iteration 290, loss = 0.02114530
Iteration 291, loss = 0.02110396
Iteration 292, loss = 0.02107927
Iteration 293, loss = 0.02095106
Iteration 294, loss = 0.02090174
Iteration 295, loss = 0.02086652
Iteration 296, loss = 0.02074135
Iteration 297, loss = 0.02065536
Iteration 298, loss = 0.02061586
Iteration 299, loss = 0.02053314
Iteration 300, loss = 0.02054363
Iteration 301, loss = 0.02052272
Iteration 

Iteration 530, loss = 0.01113192
Iteration 531, loss = 0.01112003
Iteration 532, loss = 0.01111650
Iteration 533, loss = 0.01106406
Iteration 534, loss = 0.01118014
Iteration 535, loss = 0.01109338
Iteration 536, loss = 0.01104608
Iteration 537, loss = 0.01101460
Iteration 538, loss = 0.01097842
Iteration 539, loss = 0.01112882
Iteration 540, loss = 0.01105256
Iteration 541, loss = 0.01089900
Iteration 542, loss = 0.01093325
Iteration 543, loss = 0.01094348
Iteration 544, loss = 0.01095696
Iteration 545, loss = 0.01081725
Iteration 546, loss = 0.01079489
Iteration 547, loss = 0.01092867
Iteration 548, loss = 0.01072678
Iteration 549, loss = 0.01078968
Iteration 550, loss = 0.01069220
Iteration 551, loss = 0.01080991
Iteration 552, loss = 0.01068289
Iteration 553, loss = 0.01058236
Iteration 554, loss = 0.01067224
Iteration 555, loss = 0.01061986
Iteration 556, loss = 0.01060231
Iteration 557, loss = 0.01052156
Iteration 558, loss = 0.01055898
Iteration 559, loss = 0.01049462
Iteration 

Iteration 795, loss = 0.00671186
Iteration 796, loss = 0.00672254
Iteration 797, loss = 0.00667273
Iteration 798, loss = 0.00665068
Iteration 799, loss = 0.00669683
Iteration 800, loss = 0.00664216
Iteration 801, loss = 0.00662303
Iteration 802, loss = 0.00666457
Iteration 803, loss = 0.00664312
Iteration 804, loss = 0.00665135
Iteration 805, loss = 0.00670280
Iteration 806, loss = 0.00664111
Iteration 807, loss = 0.00658282
Iteration 808, loss = 0.00656939
Iteration 809, loss = 0.00669303
Iteration 810, loss = 0.00664933
Iteration 811, loss = 0.00646729
Iteration 812, loss = 0.00658764
Iteration 813, loss = 0.00661098
Iteration 814, loss = 0.00653287
Iteration 815, loss = 0.00654122
Iteration 816, loss = 0.00664204
Iteration 817, loss = 0.00648257
Iteration 818, loss = 0.00644885
Iteration 819, loss = 0.00645416
Iteration 820, loss = 0.00646338
Iteration 821, loss = 0.00646005
Iteration 822, loss = 0.00639717
Iteration 823, loss = 0.00644279
Iteration 824, loss = 0.00637496
Iteration 

/home/lindino/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


...

Iteration 4, loss = 0.53500624
Iteration 5, loss = 0.48212798
Iteration 6, loss = 0.43728457
Iteration 7, loss = 0.39939367
Iteration 8, loss = 0.36719495
Iteration 9, loss = 0.33944930
Iteration 10, loss = 0.31573889
Iteration 11, loss = 0.29494681
Iteration 12, loss = 0.27684423
Iteration 13, loss = 0.26103421
Iteration 14, loss = 0.24681272
Iteration 15, loss = 0.23410951
Iteration 16, loss = 0.22275525
Iteration 17, loss = 0.21239112
Iteration 18, loss = 0.20298541
Iteration 19, loss = 0.19427359
Iteration 20, loss = 0.18653081
Iteration 21, loss = 0.17927900
Iteration 22, loss = 0.17260929
Iteration 23, loss = 0.16667683
Iteration 24, loss = 0.16093875
Iteration 25, loss = 0.15567814
Iteration 26, loss = 0.15076273
Iteration 27, loss = 0.14636656
Iteration 28, loss = 0.14196641
Iteration 29, loss = 0.13805559
Iteration 30, loss = 0.13425898
Iteration 31, loss = 0.13079134
Iteration 32, loss = 0.12742091
Iteration 33, loss = 0.12436462
Iteration 34, loss = 0.12124730
Iteration 35, 

Iteration 260, loss = 0.01932647
Iteration 261, loss = 0.01935984
Iteration 262, loss = 0.01931308
Iteration 263, loss = 0.01911925
Iteration 264, loss = 0.01902833
Iteration 265, loss = 0.01896547
Iteration 266, loss = 0.01889032
Iteration 267, loss = 0.01889739
Iteration 268, loss = 0.01873488
Iteration 269, loss = 0.01878511
Iteration 270, loss = 0.01862208
Iteration 271, loss = 0.01857082
Iteration 272, loss = 0.01849511
Iteration 273, loss = 0.01848554
Iteration 274, loss = 0.01834522
Iteration 275, loss = 0.01830727
Iteration 276, loss = 0.01821202
Iteration 277, loss = 0.01817702
Iteration 278, loss = 0.01811774
Iteration 279, loss = 0.01814024
Iteration 280, loss = 0.01796520
Iteration 281, loss = 0.01793784
Iteration 282, loss = 0.01790070
Iteration 283, loss = 0.01782635
Iteration 284, loss = 0.01773019
Iteration 285, loss = 0.01773708
Iteration 286, loss = 0.01760862
Iteration 287, loss = 0.01762766
Iteration 288, loss = 0.01755486
Iteration 289, loss = 0.01744432
Iteration 

Iteration 509, loss = 0.00973418
Iteration 510, loss = 0.00970400
Iteration 511, loss = 0.00973297
Iteration 512, loss = 0.00981305
Iteration 513, loss = 0.00968919
Iteration 514, loss = 0.00969238
Iteration 515, loss = 0.00966697
Iteration 516, loss = 0.00957502
Iteration 517, loss = 0.00960822
Iteration 518, loss = 0.00955610
Iteration 519, loss = 0.00955843
Iteration 520, loss = 0.00951598
Iteration 521, loss = 0.00951184
Iteration 522, loss = 0.00950089
Iteration 523, loss = 0.00950519
Iteration 524, loss = 0.00942071
Iteration 525, loss = 0.00948736
Iteration 526, loss = 0.00959760
Iteration 527, loss = 0.00945011
Iteration 528, loss = 0.00942077
Iteration 529, loss = 0.00935640
Iteration 530, loss = 0.00940691
Iteration 531, loss = 0.00932531
Iteration 532, loss = 0.00933124
Iteration 533, loss = 0.00928836
Iteration 534, loss = 0.00928129
Iteration 535, loss = 0.00932641
Iteration 536, loss = 0.00929604
Iteration 537, loss = 0.00923017
Iteration 538, loss = 0.00929036
Iteration 

Iteration 759, loss = 0.00610350
Iteration 760, loss = 0.00615940
Iteration 761, loss = 0.00609392
Iteration 762, loss = 0.00613161
Iteration 763, loss = 0.00608367
Iteration 764, loss = 0.00614905
Iteration 765, loss = 0.00611100
Iteration 766, loss = 0.00606343
Iteration 767, loss = 0.00613244
Iteration 768, loss = 0.00623284
Iteration 769, loss = 0.00609003
Iteration 770, loss = 0.00607948
Iteration 771, loss = 0.00606272
Iteration 772, loss = 0.00601915
Iteration 773, loss = 0.00599954
Iteration 774, loss = 0.00604930
Iteration 775, loss = 0.00598314
Iteration 776, loss = 0.00596551
Iteration 777, loss = 0.00598228
Iteration 778, loss = 0.00602286
Iteration 779, loss = 0.00592437
Iteration 780, loss = 0.00597365
Iteration 781, loss = 0.00593176
Iteration 782, loss = 0.00595993
Iteration 783, loss = 0.00590760
Iteration 784, loss = 0.00590857
Iteration 785, loss = 0.00593200
Iteration 786, loss = 0.00591649
Iteration 787, loss = 0.00588196
Iteration 788, loss = 0.00589661
Iteration 

/home/lindino/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## Redes Neurais com Keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

MLP_keras = Sequential()
MLP_keras.add(Dense(units = 2, activation = 'relu', input_dim = 3))
MLP_keras.add(Dense(units = 2, activation = 'relu'))
MLP_keras.add(Dense(units = 1, activation = 'sigmoid'))

MLP_keras.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
MLP_keras.fit(previsores_train, classe_train, batch_size = 10, epochs = 100)

previsoes = MLP.predict(previsores_test)
previsoes = (previsoes > 0.5)

MLP_keras_accuracy = accuracy_score(classe_test, previsoes)
matriz = confusion_matrix(classe_test, previsoes)

## Teste de Friedman

In [ ]:
from scipy.stats import friedmanchisquare
matriz = pd.read_csv('dados_estatisticos.csv')
friedmanchisquare(*[row for index, row in matriz.iterrows()])

## Combinação de classificadores

In [44]:
def paga(resultados, probabilidades, confianca_minima=0.98):
    paga = 0
    nao_paga = 0
    
    for i in range(len(resultados)):
        if probabilidades[i] >= confianca_minima:
            if resultados[i][0] == 1:
                paga += 1
            else:
                nao_paga += 1
    
    return paga, nao_paga
    
SVM = load_model('SVM')
Random_Forest = load_model('Random_Forest')
MLP = load_model('MLP')

novo_registro = np.asarray([[50000, 40, 5000]]).reshape(-1,1)
scaler = StandardScaler()
novo_registro = scaler.fit_transform(novo_registro).reshape(-1,3)

resultados = [SVM.predict(novo_registro), Random_Forest.predict(novo_registro), MLP.predict(novo_registro)]
probabilidades = [SVM.predict_proba(novo_registro).max(), Random_Forest.predict_proba(novo_registro).max(), MLP.predict_proba(novo_registro).max()]

resultado_final = paga(resultados, probabilidades)

if resultado_final[0] > resultado_final[1]:
    print('Cliente pagará o empréstimo!')
elif resultado_final[0] == resultado_final[1]:
    print('Resultado empatado!')
else:
    print('Cliente não pagaŕa o empŕestimo!')

Cliente não pagaŕa o empŕestimo!


## Resultados

In [23]:
GREEN = "\033[1;32m" 
RESET = '\033[0;0m'
NEGRITO = '\033[1m'
models = {'Base Line': base_line,'Naive Bayes': Naive_Bayes_accuracy, 'Decision Tree': Decision_Tree_accuracy,
          'Random Forest': Random_Forest_accuracy, 'KNN': KNN_accuracy, 'Logistic Regression': Logistic_Regression_accuracy,
          'SVM': SVM_accuracy, 'MLP': MLP_accuracy, 'MLP_Keras': MLP_keras_accuracy}

for model in models:
    print('{}{}{}: {}{:.1f}%{}'.format(NEGRITO,model,RESET,GREEN,(models[model] * 100), RESET))

Base Line: 87.2%
Naive Bayes: 92.5%
Decision Tree: 98.5%
Random Forest: 98.8%
KNN: 98.1%
Logistic Regression: 94.8%
SVM: 98.3%
MLP: 99.6%
